In [7]:
# ...existing code...
import yfinance as yf
from datetime import date
import os
import pandas as pd

start = date.today().replace(year=2023,month=1, day=1)
end = date.today()

# ลองหลาย ticker ที่มักใช้กับ Yahoo Finance
tickers = ['^SET.BK', '^SET', '^SETTH', '^SETI.BK']

def fetch_first_valid(tickers, start, end):
    for t in tickers:
        df = yf.download(t, start=start, end=end, progress=False)
        if not df.empty:
            print(f"ใช้ ticker: {t}")
            return df
    raise RuntimeError("ไม่พบข้อมูลจาก ticker ที่ลองทั้งหมด")

df = fetch_first_valid(tickers, start, end)
print(df.head())
# df contains Date index and columns: Open, High, Low, Close, Adj Close, Volume

# --- เพิ่มการบันทึก CSV ไม่ให้ซ้ำ ---
RAW_FILE = "data/raw/set_index.csv"
os.makedirs(os.path.dirname(RAW_FILE), exist_ok=True)

# เปลี่ยน index เป็นคอลัมน์ชื่อ 'date'
df_reset = df.reset_index()
# อาจมีชื่อคอลัมน์ต่างกันขึ้นกับเวอร์ชัน yfinance / pandas -> มาตรฐานเป็น 'Date' หรือ 'index'
if 'Date' in df_reset.columns:
    df_reset.rename(columns={'Date': 'date'}, inplace=True)
elif 'index' in df_reset.columns:
    df_reset.rename(columns={'index': 'date'}, inplace=True)
else:
    # ถ้าชื่อคอลัมน์แรกน่าจะเป็นวันที่ ให้ใช้ชื่อนั้น
    first_col = df_reset.columns[0]
    df_reset.rename(columns={first_col: 'date'}, inplace=True)

# ให้คอลัมน์ date เป็นชนิด datetime (เก็บเป็น ISO string ในไฟล์)
df_reset['date'] = pd.to_datetime(df_reset['date']).dt.date.astype(str)

# อ่านไฟล์เก่า (ถ้ามี) และต่อข้อมูลใหม่ แล้วกำจัดแถวซ้ำตามคอลัมน์ date
if os.path.exists(RAW_FILE):
    df_old = pd.read_csv(RAW_FILE, dtype=str)
    df_combined = pd.concat([df_old, df_reset], ignore_index=True, sort=False)
else:
    df_combined = df_reset

# เอาแถวซ้ำออก โดยเก็บแถวล่าสุด (keep='last') ถ้าต้องการเก็บแถวแรกให้เปลี่ยนเป็น 'first'
# df_combined = df_combined.drop_duplicates(subset=['date'], keep='last')

# เรียงตามวันที่ (ถ้ามี)
if 'date' in df_combined.columns:
    df_combined['date'] = pd.to_datetime(df_combined['date'])
    df_combined = df_combined.sort_values('date').reset_index(drop=True)
    df_combined['date'] = df_combined['date'].dt.date.astype(str)

# บันทึกไฟล์ CSV (ไม่บันทึก index)
df_combined.to_csv(RAW_FILE, index=False, encoding="utf-8-sig")

print(f"✅ Updated data saved to {RAW_FILE}")
# ...existing code...

ใช้ ticker: ^SET.BK
Price             Close         High          Low         Open   Volume
Ticker          ^SET.BK      ^SET.BK      ^SET.BK      ^SET.BK  ^SET.BK
Date                                                                   
2023-01-03  1678.969971  1683.380005  1673.290039  1675.699951  4618100
2023-01-04  1673.250000  1685.109985  1668.069946  1680.069946  5777500
2023-01-05  1663.859985  1676.819946  1659.900024  1674.810059  6321200
2023-01-06  1673.859985  1677.790039  1665.489990  1667.140015  4912600
2023-01-09  1691.119995  1693.420044  1684.900024  1685.300049  4677700
✅ Updated data saved to data/raw/set_index.csv


C:\Users\User\AppData\Local\Temp\ipykernel_3020\2939528655.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(t, start=start, end=end, progress=False)
